In [1]:
from __future__ import print_function

import pickle
from xml.dom import minidom
from bs4 import BeautifulSoup
import numpy as np

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding
import keras

from keras.callbacks import *

C:\Users\Ranet\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
vocab_size = 50000
text_len = 200
headline_len = 15

batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.

with open("DATA/training_data_proc.txt", "rb") as fp:
    target_texts, input_texts, decoder_input_data, tok_headline, tok_text, tokenizer = pickle.load(fp)
    
tokenizer.word_index['\n']

20

In [3]:
num_encoder_tokens = text_len
num_decoder_tokens = headline_len

#tok_text  = tok_text.reshape(5200, 200)
#tok_headline = tok_headline.reshape(5200, 15)
#decoder_input_data = decoder_input_data.reshape(5200, 15)
#tok_text = np.expand_dims(tok_text, -1)
#tok_headline = np.expand_dims(tok_headline, -1)
#decoder_input_data = np.expand_dims(decoder_input_data, -1)
print(tok_text.shape)
print(tok_headline.shape)
print(decoder_input_data.shape)

(5200, 200)
(5200, 15)
(5200, 15)


In [4]:
def generator(features, labels, batch_size):
    batch_features_enc = np.zeros((batch_size, num_encoder_tokens))
    batch_features_dec = np.zeros((batch_size, num_decoder_tokens))
    batch_labels = np.zeros((batch_size, num_decoder_tokens, vocab_size))
    while True:
        for i in range(batch_size):
            batch_labels[i] = to_categorical(labels[i], num_classes=vocab_size)
        yield [batch_features_enc, batch_features_dec], batch_labels

In [5]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(vocab_size, latent_dim)
encoder_embedding_outputs = encoder_embedding(encoder_inputs)
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_embedding_outputs)
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(vocab_size, latent_dim)
decoder_embedding_outputs = decoder_embedding(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_lstm_output, _, _ = decoder_lstm(decoder_embedding_outputs, initial_state=encoder_states)
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_lstm_output)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [6]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]


decoder_emb_outputs = decoder_embedding(decoder_inputs)
decoder_outputs, state_h, state_c = decoder_lstm(decoder_emb_outputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [29]:
# Run training
# model.fit([tok_text, decoder_input_data], tok_headline,
#           batch_size=batch_size,
#           epochs=epochs,
#           validation_split=0.2)

model.fit_generator(generator([tok_text, decoder_input_data], tok_headline, batch_size),
                    steps_per_epoch=tok_text.shape[0] / batch_size, nb_epoch=100)

# Save model
#model.save('s2s.h5')

C:\Users\Ranet\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  if __name__ == '__main__':
C:\Users\Ranet\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=81.25, epochs=100)`
  if __name__ == '__main__':


Epoch 1/100
82/81 [==============================] - 53s 649ms/step - loss: 3.8095
Epoch 2/100
82/81 [==============================] - 50s 604ms/step - loss: 3.5411
Epoch 3/100
82/81 [==============================] - 50s 604ms/step - loss: 3.3431
Epoch 4/100
82/81 [==============================] - 50s 611ms/step - loss: 3.2233
Epoch 5/100
82/81 [==============================] - 50s 609ms/step - loss: 3.1463
Epoch 6/100
82/81 [==============================] - 50s 609ms/step - loss: 3.0868
Epoch 7/100
82/81 [==============================] - 50s 614ms/step - loss: 3.0414
Epoch 8/100
82/81 [==============================] - 49s 602ms/step - loss: 3.0044
Epoch 9/100
82/81 [==============================] - 49s 596ms/step - loss: 2.9722
Epoch 10/100
82/81 [==============================] - 50s 605ms/step - loss: 2.9457
Epoch 11/100
82/81 [==============================] - 49s 600ms/step - loss: 2.9199
Epoch 12/100
82/81 [==============================] - 49s 602ms/step - loss: 2.8969
E

In [31]:
#model.save('DATA/100_2.7007.h5')
#model.save_weights('DATA/100_2.7007_weights.h5')

In [12]:
#model = keras.models.load_model('DATA/100_2.7007.h5')
model.load_weights('DATA/100_2.7007_weights.h5')

In [21]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = tokenizer.word_index['\n']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = list(tokenizer.word_index.keys())[list(tokenizer.word_index.values()).index(sampled_token_index)]
        decoded_sentence = ' '.join(decoded_sentence.split(' ') + [sampled_char])
        #import pdb;pdb.set_trace()
        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or len(decoded_sentence.split(' ')) > num_decoder_tokens - 1):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq[0, len(decoded_sentence.split(' ')) - 1] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence


In [26]:
for seq_index in range(5):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = tok_text[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('--------------------')
    print('### Input sentence:\n', input_texts[seq_index][0:100])
    print('\n### Decoded sentence:\n', decoded_sentence)
    print('\n### True headline:\n', target_texts[seq_index])

--------------------
### Input sentence:
 USAs New Yorgis suri täna Venemaa Föderatsiooni alaline esindaja ÜRO juures Vitali Tšurkin, teatas V

### Decoded sentence:
  kleidid kleidid kleidid kleidid rahulikum rahulikum rahulikum rahulikum kleidid hakkaks hakkaks hakkaks olevikku jagamiseks

### True headline:
  	 Suri Vene suursaadik ÜRO juures Vitali Tšurkin 
 
--------------------
### Input sentence:
 Hirmsad kaadrid Texasest näitavad, kuidas ema kallim taob ta 7-aastast poega rihmaga. Jõudu mehel ja

### Decoded sentence:
  kleidid kleidid kleidid kleidid rahulikum rahulikum rahulikum rahulikum kleidid hakkaks hakkaks hakkaks olevikku jagamiseks

### True headline:
  	 62 VALUSAT LÖÖKI: šokeeriv video, kus ema kallim tagus 7-aastast poissi rihmaga 
 
--------------------
### Input sentence:
 Eesti jalgpallikoondis kaotas novembris mäletatavasti Belgiale 1:8. Peatreener Martin Reim: "Ma ei t

### Decoded sentence:
  kleidid kleidid kleidid kleidid rahulikum rahulikum rahulikum rahul